# TODO title for non-zero decl

In [1]:
import sympy as sp
from sympy import sin, cos, tan
from sympy.abc import alpha, sigma, psi, delta, iota, theta
from analemma.algebra import frame, render, util, result

e1, e2, e3 = frame.base("e")
f1, f2, f3 = frame.planet()
n1, n2, n3 = frame.surface()
M = frame.meridian_plane() # n1^n3
s = frame.sunray()

m1, m2, m3 = frame.dial()
gn = frame.gnomon("n", zero_decl=False)

g = util.project_vector(gn, target_frame=frame.base("n"), render_frame=(n1, n2, n3))

## The Calculation

The calculation proceeds by defining $S = s \wedge g$, the plane containing the sunshine vector and the gnomon. Its magnitude is the (negative) cosine of the angle between $s$ and $g$. Then we can obtain a generalized solar hour angle $\mu$ as the angle between $S$ and $M$, given by

$$\cos(\mu) = \frac{-S\cdot M}{\sqrt{-S^2}\sqrt{-M^2}} = \frac{-S\cdot M}{\sin(\Xi)}$$

given that $M^2=-1$. In anticipation of setting $\delta =0$, I will apply $\delta$ as a subscript to these quantities for now.

In [18]:
S_delta = result.shadow_bivector(s, g)

render.align(r"S_\delta", S_delta)

<IPython.core.display.Math object>

Angle $\Xi$ between sun ray and gnomon:

In [19]:
cosXi_delta = sp.trigsimp((s|g).obj)
render.expression(r"\cos(\Xi_\delta)", cosXi_delta)

<IPython.core.display.Math object>

Cosine of hour angle $\mu$ given by

$$\sin(\Xi) \cos(\mu) = -S\cdot M$$

In [20]:
_, sinXi_cos_mu_delta = result.hour_angle_sincos(S_delta, M, zero_decl=False)

render.expression( r"\sin(\Xi_\delta) \cos(\mu_\delta)", sinXi_cos_mu_delta )

<IPython.core.display.Math object>

### A Simplifying Assumption

While $\sin(\Xi)\cos(\mu)$ is available in closed form, I have not been able to factorize $\sin(\Xi)\sin(\mu) = \sqrt{1-\sin^2(\Xi)\cos^2(\mu)}$. While we can always proceed numerically, there is a nice factorization available in the case where the gnomon's declination angle $\delta$ is zero, so that it remains in the $n_1 \wedge n_3$ plane, so I will work in that case from here.

In [21]:
gn_delta = gn
gn = gn.subs(delta, 0)
render.expression("g_n", gn)

<IPython.core.display.Math object>

In [22]:
g_delta = g
g = g.subs(delta, 0)
render.expression("g", g)

<IPython.core.display.Math object>

`g.obj` trails behind by one trigsimp

In [23]:
render.multivector(g.obj) # TODO is render.multivector needed?

<IPython.core.display.Math object>

Hack: ensure a healthy `g.obj` by reforming $g$ from the result of applying `sympy.trigsimp` to each component individually

In [24]:
g = util.update_coeffs(g)
g

(-sin(alpha)*cos(iota - theta) - sin(iota - theta)*cos(alpha)*cos(psi))*e_1 - sin(psi)*sin(iota - theta)*e_2 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*e_3

In [25]:
# eyeball check:
g.obj

-sin(alpha)*sin(iota - theta)*cos(psi)*e_3 - sin(alpha)*cos(iota - theta)*e_1 - sin(psi)*sin(iota - theta)*e_2 - sin(iota - theta)*cos(alpha)*cos(psi)*e_1 + cos(alpha)*cos(iota - theta)*e_3

### The Shadow Plane

Setting $\delta = 0$ in the shadow plane $S = s \wedge g$ (the plane containing the sunshine vector and the gnomon) gives a more manageable expression.

In [26]:
S = s^g

render.align("S", S)

<IPython.core.display.Math object>

Apply the $\delta=0$ assumption to the cosine of the angle $\Xi$ between sun ray and gnomon:

In [27]:
cosXi = sp.trigsimp((s|g).obj)
render.expression(r"\cos(\Xi)", cosXi)

<IPython.core.display.Math object>

Check $S^2 = (s\wedge g)^2 = (s\cdot g)^2 - s^2 g^2$ where $s^2 = g^2 = 1$. 

In [28]:
sg_squared = (s|g)*(s|g)
S_norm_check = (S|S) - ( sg_squared - 1 )
assert S_norm_check.obj.equals(0)
display( S_norm_check )

0

The magnitude of $S$ is given by $\sqrt{-S^2} = \sqrt{1 - (s\cdot g)^2} = \sqrt{1 - \cos^2(\Xi}) = \sin^2(\Xi)$:

In [29]:
sinXi = sp.sqrt( 1 - ((cos(alpha)*cos(sigma)*cos(psi) + sin(sigma)*sin(psi))*sin(iota-theta) + sin(alpha)*cos(sigma)*cos(iota-theta))**2 )
render.expression(r"\sin(\Xi)", sinXi)

<IPython.core.display.Math object>

In [30]:
# check
sg_check = sinXi**2 - ( 1 - sg_squared )
assert sg_check.obj.equals(0)
display( sg_check )

0

### The Hour Angle

We can now calculate the generalized solar hour angle $\mu$ (the angle between $S$ and $M$), given by $\cos(\mu) = \frac{-S\cdot M}{\sin(\Xi)}$ and also obtain a nice expression in closed form for $\sin(\mu)$.


In [31]:
sinXi_sin_mu, sinXi_cos_mu = result.hour_angle_sincos(S, M)

print(render.expression(r"\sin(\Xi) \sin(\mu)", sinXi_sin_mu))
render.expression(r"\sin(\Xi) \cos(\mu)", sinXi_cos_mu)

<IPython.core.display.Math object>


<IPython.core.display.Math object>

Check that $\sin^2(\Xi)\sin^2(\mu) + \sin^2(\Xi)\cos^2(\mu) = \sin^2(\Xi)$

In [32]:
Xi_check = sp.trigsimp( sp.expand(sinXi_sin_mu**2) + sp.expand(sinXi_cos_mu**2) - sp.expand(sinXi**2) )
assert Xi_check.equals(0)
render.multivector( Xi_check )

<IPython.core.display.Math object>

The ratio gives $\tan(\mu)$ as in the paper:

In [33]:
tan_mu = result.hour_angle_tan(S, M)

render.expression(r"\tan(\mu)", tan_mu)

<IPython.core.display.Math object>